In [1]:
%%bash
java --version

openjdk 11.0.25 2024-10-15
OpenJDK Runtime Environment (build 11.0.25+9-post-Ubuntu-1ubuntu122.04)
OpenJDK 64-Bit Server VM (build 11.0.25+9-post-Ubuntu-1ubuntu122.04, mixed mode, sharing)


In [3]:
%%bash
pip install h2o

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.8/265.8 MB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for h2o: filename=h2o-3.46.0.6-py2.py3-none-any.whl size=265859786 sha256=fcbc19b91f12bd5134a74201fa913129e36c695072cf59cd7d63492e7a29ca88
  Stored in directory: /root/.cache/pip/wheels/0c/95/eb/b449c73f478dbc4557b80170cf7f2aa6db15862877e9a63536
Successfully built h2o


In [4]:
import h2o
import pandas as pd
from h2o.automl import H2OAutoML
from sklearn.metrics import r2_score
import numpy as np

import warnings

warnings.filterwarnings("ignore")

In [5]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.25" 2024-10-15; OpenJDK Runtime Environment (build 11.0.25+9-post-Ubuntu-1ubuntu122.04); OpenJDK 64-Bit Server VM (build 11.0.25+9-post-Ubuntu-1ubuntu122.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.10/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp2tvr6hhx
  JVM stdout: /tmp/tmp2tvr6hhx/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp2tvr6hhx/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,06 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.6
H2O_cluster_version_age:,1 month and 10 days
H2O_cluster_name:,H2O_from_python_unknownUser_8z4jgx
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.170 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [6]:
df = h2o.import_file("car data.csv")

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [7]:
df = df.drop("Car_Name", axis = 1)

In [8]:
df['number_of_years'] = 2020 - df['Year']
df = df.drop("Year", axis = 1)

In [ ]:
train, test = df.split_frame(ratios=[0.8], seed=42)

target = "Selling_Price"
features = df.columns.remove(target)

automl = H2OAutoML(max_runtime_secs = 1800,
                include_algos = ["GLM", "DRF", "XGBoost"], sort_metric = "RMSE", seed = 42)

automl.train(features, target, train)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%

In [ ]:
leaderboard = h2o.automl.get_leaderboard(automl, extra_columns = "ALL")
print(leaderboard)

<table>
<thead>
<tr>
    <th>Model ID</th>
    <th>RMSE</th>
    <th>MSE</th>
    <th>MAE</th>
    <th>RMSLE</th>
    <th>Mean Residual Deviance</th>
    <th>Training Time (ms)</th>
    <th>Predict Time per Row (ms)</th>
    <th>Algorithm</th>
</tr>
</thead>
<tbody>
<tr>
    <td>DRF_grid_1_AutoML_1_20241212_54440_model_215</td>
    <td>1.28412</td>
    <td>1.64897</td>
    <td>0.662443</td>
    <td>0.142381</td>
    <td>1.64897</td>
    <td>251</td>
    <td>0.032193</td>
    <td>DRF</td>
</tr>
<tr>
    <td>DRF_grid_1_AutoML_1_20241212_54440_model_590</td>
    <td>1.34072</td>
    <td>1.79753</td>
    <td>0.708374</td>
    <td>0.148138</td>
    <td>1.79753</td>
    <td>271</td>
    <td>0.025151</td>
    <td>DRF</td>
</tr>
<tr>
    <td>XGBoost_grid_1_AutoML_1_20241212_54440_model_484</td>
    <td>1.37046</td>
    <td>1.87816</td>
    <td>0.685618</td>
    <td>0.154746</td>
    <td>1.87816</td>
    <td>244</td>
    <td>0.013404</td>
    <td>XGBoost</td>
</tr>
<tr>
    <td>XGBoost_grid_1_AutoML_1_20241212_54440_model_266</td>
    <td>1.41737</td>
    <td>2.00895</td>
    <td>0.715006</td>
    <td>0.153616</td>
    <td>2.00895</td>
    <td>302</td>
    <td>0.01228</td>
    <td>XGBoost</td>
</tr>
<tr>
    <td>DRF_grid_1_AutoML_1_20241212_54440_model_596</td>
    <td>1.41977</td>
    <td>2.01575</td>
    <td>0.714688</td>
    <td>0.149661</td>
    <td>2.01575</td>
    <td>285</td>
    <td>0.010096</td>
    <td>DRF</td>
</tr>
<tr>
    <td>DRF_grid_1_AutoML_1_20241212_54440_model_467</td>
    <td>1.44677</td>
    <td>2.09315</td>
    <td>0.690772</td>
    <td>0.149236</td>
    <td>2.09315</td>
    <td>496</td>
    <td>0.009819</td>
    <td>DRF</td>
</tr>
<tr>
    <td>DRF_grid_1_AutoML_1_20241212_54440_model_297</td>
    <td>1.4898</td>
    <td>2.21951</td>
    <td>0.784992</td>
    <td>nan</td>
    <td>2.21951</td>
    <td>286</td>
    <td>0.010156</td>
    <td>DRF</td>
</tr>
<tr>
    <td>DRF_grid_1_AutoML_1_20241212_54440_model_454</td>
    <td>1.49179</td>
    <td>2.22544</td>
    <td>0.715708</td>
    <td>0.180872</td>
    <td>2.22544</td>
    <td>227</td>
    <td>0.009188</td>
    <td>DRF</td>
</tr>
<tr>
    <td>XGBoost_grid_1_AutoML_1_20241212_54440_model_148</td>
    <td>1.50352</td>
    <td>2.26056</td>
    <td>0.662147</td>
    <td>nan</td>
    <td>2.26056</td>
    <td>237</td>
    <td>0.01005</td>
    <td>XGboost</td>
</tr>
<tr>
    <td>DRF_grid_1_AutoML_1_20241212_54440_model_418</td>
    <td>1.53849</td>
    <td>2.36695</td>
    <td>0.732047</td>
    <td>0.154009</td>
    <td>2.36695</td>
    <td>295</td>
    <td>0.01432</td>
    <td>DRF</td>
</tr>
</tbody>
</table>
